## SpaceY Dash

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px

In [3]:
# OP 1 - Create a dash application
# app = dash.Dash(__name__)
# OP 2 - JupyterDash
from jupyter_dash import JupyterDash
app = JupyterDash(__name__)

In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
spacex_df.head(20)

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,Success,0,3170.0,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,Success,0,3325.0,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,Success,0,2296.0,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,Success,0,1316.0,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,Success,0,4535.0,F9 v1.1,v1.1


In [6]:
site_list = list(set(spacex_df['Launch Site'].values))
site_list.append('All Sites')
site_list

['CCAFS SLC-40', 'VAFB SLC-4E', 'CCAFS LC-40', 'KSC LC-39A', 'All Sites']

In [7]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', value='All Sites',
                                                     # Update dropdown values using list comphrehension
                                                     options=[{'label': i, 'value': i} for i in site_list],
                                                     placeholder="Select a launch site...",
                                                     searchable=True,
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

In [8]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
                Output(component_id='success-pie-chart', component_property='figure')
                , [
                Input(component_id='site-dropdown', component_property='value')
                ]
              )
def get_pie_graph(site):
    if site == 'All Sites':
        div_data = spacex_df
        pie_fig = px.pie(div_data, values='class', names='Launch Site', title='Total Success Launches By Site')
    else:        
        div_data = spacex_df[spacex_df['Launch Site']==site]
        pie_fig = px.pie(div_data, values='Launch Site', names='class', title='Total Success Launches for site ')
    return pie_fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
                Output(component_id='success-payload-scatter-chart', component_property='figure')
                , [
                Input(component_id='site-dropdown', component_property='value'), 
                Input(component_id="payload-slider", component_property="value")
                ]
              )
def get_scatter_graph(site, slider):
    if site == 'All Sites':
        line_data = spacex_df
        line_data = line_data[spacex_df['Payload Mass (kg)']>=slider[0]]
        line_data = line_data[spacex_df['Payload Mass (kg)']<slider[1]]
        line_fig = px.scatter(line_data, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Correlation bteween Payload and Success for all Sites')
    else:        
        line_data = spacex_df[spacex_df['Launch Site']==site]
        line_data = line_data[spacex_df['Payload Mass (kg)']>=slider[0]]
        line_data = line_data[spacex_df['Payload Mass (kg)']<slider[1]]
        line_fig = px.scatter(line_data, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Correlation bteween Payload and Success for all Sites')
    return line_fig


In [9]:
# OP 1 - Run the server
#if __name__ == '__main__':
#    app.run_server()
# OP 2 - JupyterDash
#app.run_server(mode='inline', host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)
app.run_server(mode='inline', host="localhost", debug=False)

 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Aug/2021 19:53:04] "GET /_alive_3c46ccc3-d454-4b4c-b4c7-be1910f7b56f HTTP/1.1" 200 -
